<a href="https://colab.research.google.com/github/wenjunc2/PAHO-Drug-Data-Cleaning-Summer-2022/blob/main/%E6%99%AE%E9%80%9A%E7%BC%96%E7%A8%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1: Set up indexes for clean catagol drug names

In [ ]:
# read into clean catagol name
import pandas as pd
CleanDataBase = pd.read_excel('CleanDataBase.xlsx')
CatalogName = CleanDataBase['Catalog Name']
CatalogName = list(set(CatalogName))
CatalogName = [x for x in CatalogName if str(x) != 'nan']

CatalogNameCopy = CatalogName.copy()
# uppercase everything
CatalogNameU = []
for i in CatalogNameCopy:
  CatalogNameU.append(i.upper())

In [ ]:
CatalogNameU[:5]

['CALCIUM GLUCONATE 100 MG/ML, INJECTION, 10 ML AMPOULE, TBD',
 'ACICLOVIR 250 MG, POWDER FOR INJECTION, VIAL, 5X1',
 'AMLODIPINO 5 MG + HIDROCLOROTIAZIDA 25 MG + VALSARTAN 160 MG, TABLET, TBD',
 'CANDESARTAN 32 MG, TABLET, TBD',
 'CAPECITABINE 500 MG, TABLET, BLISTER, 10X12']

In [ ]:
# split the CatalogNameU
#from nltk.tokenize import WordPunctTokenizer  # this function split the string by space
#tokenizer = WordPunctTokenizer()

CatalogNameUS = []
for item in CatalogNameU:
  temp = item.split(",")
  CatalogNameUS.append(temp)

In [ ]:
CatalogNameUS[:10] # after split by ","

[['CALCIUM GLUCONATE 100 MG/ML', ' INJECTION', ' 10 ML AMPOULE', ' TBD'],
 ['ACICLOVIR 250 MG', ' POWDER FOR INJECTION', ' VIAL', ' 5X1'],
 ['AMLODIPINO 5 MG + HIDROCLOROTIAZIDA 25 MG + VALSARTAN 160 MG',
  ' TABLET',
  ' TBD'],
 ['CANDESARTAN 32 MG', ' TABLET', ' TBD'],
 ['CAPECITABINE 500 MG', ' TABLET', ' BLISTER', ' 10X12'],
 ['LISINOPRIL 20 MG', ' TABLET', ' BLISTER', ' 10X3'],
 ['CISPLATIN 50 MG', ' POWDER FOR INJECTION', ' 50 ML VIAL', ' 1X1'],
 ['VALSARTAN 80 MG + HYDROCHLOROTHIAZIDE 12.5 MG', ' TABLET', ' TBD'],
 ['ROSUVASTATIN 20 MG', ' TABLET', ' TBD'],
 ['AZITHROMYCIN 500 MG', ' TABLET', ' BLISTER', ' 3X1']]

In [ ]:
# summary the list of presentation form
PresentationForm = []

for item in CatalogNameUS:
  if len(item) >= 2:
    PresentationForm.append(item[1])


In [ ]:
PresentationForm = list(set(PresentationForm))

len(PresentationForm)

29

In [ ]:
PresentationForm[:10] # the presentation form

[' 50MG TABLET',
 ' CAPSULE (PRONLONGED RELEASE)',
 ' SOLUTION FOR INJECTION',
 '\xa0TABLET',
 ' POWDER FOR INJECTION',
 ' TABLET (COATED)',
 ' INJECTION. 500MG/20ML ROAD BOTTLE',
 ' CAPSULE',
 ' TABLET (DISPERSIBLE',
 ' POWDER FOR INJECTION. 500MCG ROAD BOTTLE']

In [ ]:
# grab the main medicine 
MainMedicine = []

for item in CatalogNameUS:
  MainMedicine.append(item[0])

In [ ]:
print(len(MainMedicine), MainMedicine[:5])

462 ['CALCIUM GLUCONATE 100 MG/ML', 'ACICLOVIR 250 MG', 'AMLODIPINO 5 MG + HIDROCLOROTIAZIDA 25 MG + VALSARTAN 160 MG', 'CANDESARTAN 32 MG', 'CAPECITABINE 500 MG']


In [ ]:
# split MainMedicine to get only the medicne name, and use the medicine name as the index
MainMedicineSpl = []
for item in MainMedicine:
  tokenlist = tokenizer.tokenize(item)
  MainMedicineSpl.append(tokenlist)

In [ ]:
MainMedicineSpl[:5]

[['CALCIUM', 'GLUCONATE', '100', 'MG', '/', 'ML'],
 ['ACICLOVIR', '250', 'MG'],
 ['AMLODIPINO',
  '5',
  'MG',
  '+',
  'HIDROCLOROTIAZIDA',
  '25',
  'MG',
  '+',
  'VALSARTAN',
  '160',
  'MG'],
 ['CANDESARTAN', '32', 'MG'],
 ['CAPECITABINE', '500', 'MG']]

In [ ]:
MainMedicineIndex = []
for item in MainMedicineSpl:
  templist = []
  for subitem in item:
    if len(subitem) > 4 and subitem.isdigit() is False:
      templist.append(subitem)
  MainMedicineIndex.append(templist)


In [ ]:
MainMedicineIndex[:5]

[['CALCIUM', 'GLUCONATE'],
 ['ACICLOVIR'],
 ['AMLODIPINO', 'HIDROCLOROTIAZIDA', 'VALSARTAN'],
 ['CANDESARTAN'],
 ['CAPECITABINE']]

In [ ]:
CatalogIndexData = {"Catalog Name": CatalogNameU, "Catalog Index": MainMedicineIndex}

In [ ]:
CatalogIndexData_df = pd.DataFrame(CatalogIndexData)

In [53]:
len(CatalogIndexData_df)

462

In [59]:
CatalogIndexData_df['Catalog Index']

0                            [CALCIUM, GLUCONATE]
1                                     [ACICLOVIR]
2      [AMLODIPINO, HIDROCLOROTIAZIDA, VALSARTAN]
3                                   [CANDESARTAN]
4                                  [CAPECITABINE]
                          ...                    
457                      [METFORMIN, GLIMEPIRIDE]
458                                  [METOPROLOL]
459                    [GLIBENCLAMIDE, METFORMIN]
460                                   [DELAMANID]
461                      [SITAGLIPTIN, METFORMIN]
Name: Catalog Index, Length: 462, dtype: object

In [55]:
CatalogIndexData_df

,Catalog Name,Catalog Index
0,"CALCIUM GLUCONATE 100 MG/ML, INJECTION, 10 ML ...","[CALCIUM, GLUCONATE]"
1,"ACICLOVIR 250 MG, POWDER FOR INJECTION, VIAL, 5X1",[ACICLOVIR]
2,AMLODIPINO 5 MG + HIDROCLOROTIAZIDA 25 MG + VA...,"[AMLODIPINO, HIDROCLOROTIAZIDA, VALSARTAN]"
3,"CANDESARTAN 32 MG, TABLET, TBD",[CANDESARTAN]
4,"CAPECITABINE 500 MG, TABLET, BLISTER, 10X12",[CAPECITABINE]
...,...,...
457,"METFORMIN 1000 MG + GLIMEPIRIDE 4 MG, TABLET, TBD","[METFORMIN, GLIMEPIRIDE]"
458,"METOPROLOL 100 MG, TABLET, BLISTER, 10X3",[METOPROLOL]
459,"GLIBENCLAMIDE 5 MG + METFORMIN 500 MG, TABLET,...","[GLIBENCLAMIDE, METFORMIN]"
460,"DELAMANID 50 MG, TABLET, BLISTER, 28X24",[DELAMANID]
